In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>, Cristovao da Cruz e Silva
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt

os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)


from scripts.log_action import log_action_v2

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
# chip_names = ["ET2_Bare_2", "ET2_Bare_3"]
# chip_fignames = ["ETROC 2.01 Bare Board 2", "ETROC 2.01 Bare Board 3"]

chip_names = ["ET2p01_Bare_5"]
chip_fignames = ["ETROC 2.01 Bare Board 5"]

chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM1"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60]
#chip_addresses = [0x60]
ws_addresses = [0x40, None]

# fig_outdir = Path('../ETROC-figures')
fig_outdir = Path('/media/daq/X9/SEUApril2024/ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# config_outdir = Path('../ETROC-Data')
config_outdir = Path('/media/daq/X9/SEUApril2024/ETROC-Data')
config_outdir = config_outdir / "ChipConfig"
config_outdir.mkdir(exist_ok=True)

histdir = Path('/media/daq/X9/SEUApril2024/ETROC-History')
histdir.mkdir(exist_ok=True)

log_path = Path(f'/home/{os.getlogin()}/ETROC2/i2c_gui')

full_col_list, full_row_list = np.meshgrid(np.arange(16),np.arange(16))
full_scan_list = list(zip(full_row_list.flatten(),full_col_list.flatten()))

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1"),("1","1")])

# Config chips

## Key is (WS Init, All Offsets 20 , Cal+Disable, Auto Cal, Disable Pixels, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
# (WS Prep Pixel and Peri) - 0 - (disable & auto_cal all pixels) - (auto_TH_CAL) - (disable default all pixels) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
log_action_v2(log_path, "DAQ", "Config", "Initial")
i2c_conn.config_chips('00001101')
for address in chip_addresses:
    log_action_v2(log_path, "DAQ", "Config", "High Power")
    i2c_conn.set_power_mode_scan_list(address, full_scan_list, 'high')
log_action_v2(log_path, "DAQ", "Config", "Baseline")
i2c_conn.config_chips('00010000')
log_action_v2(log_path, "DAQ", "Config", "Done")

## Dump I2C Config


In [ ]:
i2c_conn.dump_config(config_outdir, "FirstSetup_April24_BeforeRuns")

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
def make_baselines(do_config = True, power_mode = 'high', calibrate_chip=True, calibrate_pixels=None):
    if do_config:
        log_action_v2(log_path, "DAQ", "Config", "Basic and Disable")
        i2c_conn.config_chips('00001100')
        for address in chip_addresses:
            if power_mode == 'low':
                log_action_v2(log_path, "DAQ", "Config", "Low Power")
            elif power_mode == 'high':
                log_action_v2(log_path, "DAQ", "Config", "High Power")
            else:
                log_action_v2(log_path, "DAQ", "Config", "Set Power")
            i2c_conn.set_power_mode_scan_list(address, full_scan_list, power_mode = power_mode)

    # histdir = Path('/media/daq/X9/SEUApril2024/ETROC-History-SEUEquivalence')
    # histdir.mkdir(exist_ok=True)
    histfile = histdir / 'BaselineHistory.sqlite'

    log_action_v2(log_path, "DAQ", "Config", "Baseline")
    if(calibrate_chip): i2c_conn.config_chips('00010000')
    elif(calibrate_pixels is not None):
        data = []
        for aidx,address in enumerate(chip_addresses):
            for row,col in calibrate_pixels:
                i2c_conn.auto_cal_pixel(chip_name=chip_names[aidx], row=row, col=col, verbose=True, chip_address=address, data=data)
        BL_df = pandas.DataFrame(data = data)
        with sqlite3.connect(histfile) as sqlconn:
            BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)
        del data, BL_df
        return
    
    log_action_v2(log_path, "DAQ", "Config", "Done")

    # from mpl_toolkits.axes_grid1 import make_axes_locatable
    for chip_address, chip_figname, chip_figtitle in zip(chip_addresses,chip_fignames,chip_fignames):
        BL_map_THCal,NW_map_THCal,BL_df,_ = i2c_conn.get_auto_cal_maps(chip_address)
        fig = plt.figure(dpi=200, figsize=(20,10))
        gs = fig.add_gridspec(1,2)

        ax0 = fig.add_subplot(gs[0,0])
        ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=17, loc="right")
        img0 = ax0.imshow(BL_map_THCal, interpolation='none')
        ax0.set_aspect("equal")
        ax0.invert_xaxis()
        ax0.invert_yaxis()
        plt.xticks(range(16), range(16), rotation="vertical")
        plt.yticks(range(16), range(16))
        hep.cms.text(loc=0, ax=ax0, fontsize=17, text="Preliminary")
        divider = make_axes_locatable(ax0)
        cax = divider.append_axes('right', size="5%", pad=0.05)
        fig.colorbar(img0, cax=cax, orientation="vertical")

        ax1 = fig.add_subplot(gs[0,1])
        ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=17, loc="right")
        img1 = ax1.imshow(NW_map_THCal, interpolation='none')
        ax1.set_aspect("equal")
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        plt.xticks(range(16), range(16), rotation="vertical")
        plt.yticks(range(16), range(16))
        hep.cms.text(loc=0, ax=ax1, fontsize=17, text="Preliminary")
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes('right', size="5%", pad=0.05)
        fig.colorbar(img1, cax=cax, orientation="vertical")

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

        for x in range(16):
            for y in range(16):
                ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
                ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
        plt.tight_layout()
        plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+timestamp+".png")
        plt.show()

        with sqlite3.connect(histfile) as sqlconn:
            BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)
        # savetxt(histdir / f'{chip_figname}_BL_{timestamp}.csv', BL_map_THCal, delimiter=',')
        # savetxt(histdir / f'{chip_figname}_NW_{timestamp}.csv', NW_map_THCal, delimiter=',')

    time.sleep(1)

In [ ]:
# #%%
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.figure()
# plt.show()

In [ ]:
make_baselines(True)

# Define Utility Functions

In [ ]:
def do_pixel_operations(scan_list, board_offsets=None, noisy_pixels=None, power_mode='high', Bypass_THCal=True, QInjEn=True):
    log_action_v2(log_path, "DAQ", "Config", "Pixel Operations")
    i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=QInjEn,Bypass_THCal=Bypass_THCal,specified_addresses=chip_addresses[:], power_mode=power_mode, verbose=False)

    if board_offsets is not None:
        time.sleep(1)
        for chip_address in chip_addresses[:]:
            if chip_address not in board_offsets:
                continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            if(Bypass_THCal): i2c_conn.set_chip_offsets(chip_address, offset=board_offsets[chip_address], chip=chip, pixel_list=scan_list, verbose=True)
            for row, col in scan_list:
                print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col}) for QSel and maybe Setting Offset to {board_offsets[chip_address]}")
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)
                # i2c_conn.pixel_decoded_register_write("QSel", format(0x1e, '05b'), chip) # 30 fC
                # i2c_conn.pixel_decoded_register_write("QInjEn", "1" if QInjEn else "0", chip)
                if(not Bypass_THCal): i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
            del chip, row_indexer_handle, column_indexer_handle

    if noisy_pixels is not None:
        time.sleep(1)
        for chip_address in chip_addresses:
            if chip_address not in noisy_pixels:
                continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            for row, col in noisy_pixels[chip_address]:
                print(f"Masking from trigger of chip {hex(chip_address)} Pixel ({row},{col})")
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)
                i2c_conn.pixel_decoded_register_write("disTrigPath", "1", chip)
                #i2c_conn.pixel_decoded_register_write("DAC", format(1023, '010b'), chip)
                #i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", chip)

    time.sleep(1)

In [ ]:
def do_all_pix_operations(Bypass_THCal=True):
    log_action_v2(log_path, "DAQ", "Config", "Pixel Operations")
    for chip_address in chip_addresses[:]:
        chip = i2c_conn.get_chip_i2c_connection(chip_address)
        row_indexer_handle,_,_ = chip.get_indexer("row")
        column_indexer_handle,_,_ = chip.get_indexer("column")
        broadcast_handle,_,_ = chip.get_indexer("broadcast")
        column_indexer_handle.set(0)
        row_indexer_handle.set(0)
        chip.read_all_block("ETROC2", "Pixel Config")
        Bypass_THCal_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "Bypass_THCal")
        TH_offset_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "TH_offset")
        DAC_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "DAC")
        Bypass_THCal_handle.set("1" if Bypass_THCal else "0")
        TH_offset_handle.set(hex(0x3f))  # Max Offset 63
        DAC_handle.set(hex(0x3ff)) # Max DAC 1023
        broadcast_handle.set(True)
        chip.write_all_block("ETROC2", "Pixel Config")
        print(f"{Bypass_THCal} Bypass_THCal and Set TH_Offset to 0x3f and DAC to 0x3ff for all pixels of chip: {hex(chip_address)}")

In [ ]:
def do_pixel_operations_checkerboard(scan_list, full_scan_list_bool, board_offsets=None, power_mode='high'):
    log_action_v2(log_path, "DAQ", "Config", "Pixel Operations")
    i2c_conn.enable_select_pixels_in_chips(scan_list,QInjEn=True,Bypass_THCal=True,specified_addresses=chip_addresses[:], power_mode=power_mode, verbose=False)
    if board_offsets is not None:
        for chip_address in chip_addresses[:]:
            if chip_address not in board_offsets: continue
            chip = i2c_conn.get_chip_i2c_connection(chip_address)
            row_indexer_handle,_,_ = chip.get_indexer("row")
            column_indexer_handle,_,_ = chip.get_indexer("column")
            for row,col in tqdm(full_scan_list_bool.keys()):
                pxflag = ((row+col)%2)==0
                if(not (pxflag or full_scan_list_bool[(row,col)])): continue
                column_indexer_handle.set(col)
                row_indexer_handle.set(row)
                if(pxflag):
                    # chip.read_all_block("ETROC2", "Pixel Config")
                    # Bypass_THCal_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "Bypass_THCal")
                    # Bypass_THCal_handle.set("0")
                    i2c_conn.pixel_decoded_register_write("Bypass_THCal", "0", chip)
                    if(full_scan_list_bool[(row,col)]):
                        # print(f"Pix {row},{col} Offset method")
                        # TH_offset_handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", "TH_offset")
                        # TH_offset_handle.set(hex(board_offsets[chip_address]))
                        i2c_conn.pixel_decoded_register_write("TH_offset", format(board_offsets[chip_address], '06b'), chip)
                    # chip.write_all_block("ETROC2", "Pixel Config")
                else:
                    # print(f"Pix {row},{col} DAC method")
                    i2c_conn.set_pixel_offsets(chip_address, row, col, offset=board_offsets[chip_address], chip=chip, verbose=False, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
            del chip, row_indexer_handle, column_indexer_handle
    time.sleep(1)

In [ ]:
self_trigger_delay = 484
#LED_pages = ["0x0000","0x0004","0x0008","0x000C"] # LED pages to cycle through
LED_pages = ["0x0000"] # LED pages to cycle through
active_channels = "0x0001"
polarity = "0xC023" # or "0x0027"
hostname = "192.168.2.3"

In [ ]:
def run_daq(trigger_board, trigger_board_name, run_number, total_time, daq_time, run_options="--compressed_binary --skip_translation -v", allow_overwrite = False, reset_string = "--check_valid_data_start --clear_fifo"):
    out_dir = Path('/media/daq/X9/SEUApril2024/ETROC-Data')
    if not out_dir.exists():
        raise RuntimeError("The base output directory does not exist... there may be a script config problem")
    if not allow_overwrite:
        run_dir = out_dir / f"Run_{run_number}"
        if run_dir.exists():
            raise RuntimeError("Run directory already exists, I am not going to overwrite")

    # trigger_bit_delay = int(trigger_board+'11'+format(self_trigger_delay, '010b'), base=2)
    trigger_bit_delay = 0x1800
    process_time = daq_time + 2
    iterations = ceil(total_time/process_time)
    LED_id = 0

    for it in range(iterations):
        LED_page = LED_pages[LED_id]
        LED_id += 1
        if LED_id >= len(LED_pages): LED_id = 0
        reset_string_used = ""
        if(it==0): reset_string_used = reset_string
        parser = parser_arguments.create_parser()
        (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t {process_time} --run_name Run_{run_number} -o loop_{it} -w -s {LED_page} -p {polarity} -d {trigger_bit_delay} -a {active_channels} {run_options} --ssd --ssd_path {out_dir.absolute().as_posix()} --start_DAQ_pulse --stop_DAQ_pulse {reset_string_used}".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Run_{run_number}_loop_{it}'))
        process.start()

        IPC_queue.put('memoFC Start Triggerbit BCR L1A QInj repeatedQInj=67 L1ARange')
        while not IPC_queue.empty():
            pass
        time.sleep(daq_time)
        IPC_queue.put('stop DAQ')
        IPC_queue.put('memoFC Triggerbit')
        while not IPC_queue.empty():
            pass
        IPC_queue.put('allow threads to exit')
        process.join()

        del IPC_queue, process, parser

# Run DAQ (Actual SEU Data Taking Starts Here)

In [ ]:
trigger_board = '0001'
trigger_board_name = chip_names[0]

col_list = [8, 2, 8, 2]
row_list = [0, 0, 2, 2]
# col_list = [8, 2, 8]
# row_list = [0, 0, 2]
# col_list = [8, 2]
# row_list = [0, 0]
# col_list = [8]
# row_list = [0]
#col_list = []
#row_list = []
scan_list = list(zip(row_list, col_list))
full_scan_list_bool = {(row,col):False for row,col in full_scan_list}
for row,col in scan_list:
    full_scan_list_bool[(row,col)] = True

board_offsets = {
    0x60: 0x14,
    # 0x61: 0x14,
    #0x62: 0x0f,
    #0x64: 0x0a,
}
noisy_pixels = {
    #0x60: [ # (row, col)
    #    (10, 8),
    #],
}
power_mode = 'high'

## Pixel Operations

In [ ]:
make_baselines(do_config = True)
# make_baselines(do_config = True, power_mode = power_mode, calibrate_chip=False, calibrate_pixels=scan_list)
do_all_pix_operations(Bypass_THCal=True)
# do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode, all_pix_en_Bypass=True)
do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode, Bypass_THCal=True, QInjEn=True)
# do_pixel_operations_checkerboard(scan_list, full_scan_list_bool, board_offsets=board_offsets, power_mode='high')

In [ ]:
current_run_name = "Apr24_13"

## Short Run for testing

In [ ]:
test_run_name = f"Testrun_{current_run_name}"
# "Test_short_Apr24_512prescale_25msfiller_repQInj_Checker_2"
# run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number="Test_short_Jan27_1", total_time=1, daq_time=30, allow_overwrite=True, run_options="--compressed_binary -v", reset_string="--check_valid_data_start --clear_fifo")
log_action_v2(log_path, "DAQ", "Run", "Test Start")
run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=test_run_name, total_time=1, daq_time=10, allow_overwrite=True, run_options="--skip_binary -v", reset_string="--check_valid_data_start")
log_action_v2(log_path, "DAQ", "Run", "Test Stop")

## Dump I2C Config Before Run


In [ ]:
i2c_conn.dump_config(config_outdir, "PreRun")

## Do Run

In [ ]:
run_numbers = [current_run_name]
total_time = 960
daq_time = 60

for run_number in run_numbers:
    print(f"Run {run_number} started at:", datetime.datetime.now().isoformat(sep=" "))
    run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=run_number, total_time=total_time, daq_time=daq_time, run_options="--compressed_binary --skip_translation", reset_string="--check_valid_data_start")
    print(f"Run {run_number} ended at:", datetime.datetime.now().isoformat(sep=" "))

## Dump I2C Config After Run


In [ ]:
i2c_conn.dump_config(config_outdir, "PostRun")

# Calibrate FC for all I2C

In [ ]:
time.sleep(1)
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)
time.sleep(1)
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)
time.sleep(1)
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# SEU Equivalence
(only for running long tests outside of SEU)

In [ ]:
# total_runs = 40
# each_run_time = 1
# each_daq_time = 60*3  # Average of 3 minutes per run from Jan24

# for run_no in range(total_runs):
#     make_baselines(do_config = True, power_mode = power_mode, calibrate_chip=False, calibrate_pixels=scan_list)
#     do_pixel_operations(scan_list, board_offsets = board_offsets, noisy_pixels = noisy_pixels, power_mode = power_mode)

#     time.sleep(60)

#     i2c_conn.dump_config(config_outdir, "PreRun")

#     time.sleep(20)

#     run_name = f"SEUEquivalenceFeb13_{run_no}"

#     log_action_v2(log_path, "DAQ", "Run", "Start")
#     print(f"Run {run_name} started at:", datetime.datetime.now().isoformat(sep=" "))
#     run_daq(trigger_board=trigger_board, trigger_board_name=trigger_board_name, run_number=run_name, total_time=each_run_time, daq_time=each_daq_time, run_options="--compressed_binary --skip_translation", reset_string="--check_valid_data_start")
#     print(f"Run {run_name} ended at:", datetime.datetime.now().isoformat(sep=" "))
#     log_action_v2(log_path, "DAQ", "Run", "Start")

#     i2c_conn.dump_config(config_outdir, "PostRun")

#     time.sleep(10)